## Importing libs

In [1]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [2, 2]
sns.set() # apply the seaborn defaults to plotted figures (e.g. theme, scaling, color palette), instead of matplotlib's

## Functions and derivatives

In [3]:
def linear(z):
    return z

def relu(z):
    return np.maximum(0, z)

def sigmoid(z, limit=500):
    if limit != None:
        z = np.clip(z, -limit, limit) # avoid overflow
    return 1 / (1 + np.exp(-z))

def softmax(y_pred, axis=-1):
    exp = np.exp(y_pred)
    return exp / np.sum(exp, axis=axis, keepdims=True)

''' y.shape == y_pred.shape == (m, C), where:
    - m is the number of examples
    - C is the number of classes 
    Thus, each row of y and y_pred is a one-hot encoded vector of shape (1, C)
'''    

def cross_entropy(y, y_pred, axis=-1, eps=1e-12):
    if eps != None:
        y_pred = np.clip(y_pred, eps, 1 - eps) # avoid overflow
    m = y_pred.shape[0]
    return -np.sum(y * log(y_pred), axis=axis) / m

def xent(y, y_pred):
    return -np.sum(y * log(y_pred))

In [4]:
def grad_linear(z):
    return np.ones(shape=z.shape)

def grad_relu(z):
    return np.where(z > 0, 1, 0)

def grad_sigmoid(z):
    sigmoid_z = sigmoid(z)
    return sigmoid_z * (1 - sigmoid_z)

def grad_softmax(y_pred):
    # y_pred[i]*(1-y_pred[j]) if i != j --> y_pred[i] - y_pred[i] * y_pred[j]
    # -y_pred[i]*y_pred[j]    if i == j -->     0     - y_pred[i] * y_pred[j]
    y_pred = y_pred.reshape(-1, 1)
    return np.diagflat(y_pred) - np.dot(y_pred, y_pred.T)

def grad_cross_entropy(y, y_pred, axis=-1):
    return y_pred - y # FIXME

## Neural Network

From a layer $k-1$ to a layer $k$ we have:
- Weights $W \in \mathbb{R}^{n_k \times n_{k-1}}$
- Biases $b \in \mathbb{R}^{n_k}$
- Activations $a^{(k)} = g_k(z^{(k)}) \in \mathbb{R}^{n_k}$, where $g_k(z^{(k)})$ is the activation function of the $k^{\text{th}}$ layer and $z^{(k)} = W^{(k)} a^{(k-1)} + b^{(k)}$

(Xavier initialization: [[1]](https://prateekvjoshi.com/2016/03/29/understanding-xavier-initialization-in-deep-neural-networks/))

For the first layer, the activation is the input itself: $a^{(1)} = x$  
For the middle layers ($2 \leq k < L$), the activation function is the sigmoid: $a^{(k)} = g_k(z^{(k)}) = sigmoid(z^{(k)})$  
For the last layer, we have the predicted value with softmax activation: $a^{(L)} = g_k(z^{(L)}) = softmax(z^{(L)})$  
(i.e. the hypothesis function $a^{(L)} = h_{W, b}(x) = y_{\text{pred}} \approx y$)

obs.: the number of layers $L$ comes from: $1$ input layer + $1$ output layer + $L-2$ hidden layers

In [6]:
import warnings

# RANDOM_SEED = 886

In [ ]:
class Layer:    
    def __init__(self, input_size, output_size, 
                 activation_function, activation_function_derivative, 
                 weight_initialization='xavier'):
        
        self.input_size  = input_size
        self.output_size = output_size
        
        self.g = activation_function
        self.g_prime = activation_function_derivative
        
        # activation values
        self.a = None # np.array(shape=(output_size, 1))
        self.z = None # np.array(shape=(output_size, 1))
        
        if weight_initialization == 'xavier':
            stddev = np.sqrt(1 / input_size)
            self.W = stddev * np.random.randn(output_size, input_size)
            self.b = np.random.randn(output_size, 1)
        elif weight_initialization == 'xavier_avg':
            stddev = np.sqrt(2 / (input_size + output_size))
            self.W = stddev * np.random.randn(output_size, input_size)
            self.b = np.random.randn(output_size, 1)
        else:
            raise ValueError(f"Invalid weight_initialization value: '{weight_initialization}'")

class NN:
    def __init__(self, cost_function, optimizer=None):
        self.layers = []
    
    def add_layer(self, layer):
        self.layers.append(layer)
        
    def predict(self, X):
        activation = X
        for l in range(1, L):
            z = self.layers[l].W @ activation + self.layers[l].b
            activation = self.layers[l].g(z)
        return activation
    
    def feed_forward(self, X):
        L = len(self.layers)
        # note that we use zero-based indexing here, so
        # the 1st layer is self.layers[0] and the last is self.layers[L - 1]
        self.layers[0].a = X # input
        for l in range(1, L):
            self.layers[l].z = self.layers[l].W @ self.layers[l-1].a + self.layers[l].b
            self.layers[l].a = self.layers[l].g(self.layers[l].z) # apply the activation function g to the weighted sum z
        y_pred = self.layers[L-1] # output
        return y_pred
    
    def backprop(self, X, y, learning_rate):
        L = len(self.layers) # number of layers
        m = X.shape[0]       # number of examples
        
        y_pred = self.feed_forward(x) # == self.layers[L-1].a
        cost = self.J(y, y_pred)
        
        self.layers[L-1].error = y_pred - y
        self.layers[L-1].delta = self.layers[L-1].error * self.layers[L-1].g_prime(y_pred)
        for l in reversed(range(L-1)):
            self.layers[l].error = (self.layers[l+1].W).T @ self.layers[l+1].delta
            self.layers[l].delta = self.layers[l].error * self.layers[l].g_prime(self.layers[l].a)
        
        # TODO gradient descent
        
        pass
    
    def train(self, x, y):
        #TODO
        pass
            
    # TODO fit(trainning data), evaluate(validation data)
        